In [1]:
import numpy as np
import glob
import os

In [2]:
def load_asc(asc_file):
    return np.loadtxt(asc_file, skiprows=6)

In [3]:
def combine_features(raster_folder):
    return np.array([load_asc(raster_folder+"/"+raster_folder.split('/')[-1]+"_"+str(i)+".asc") for i in range(1, 9)])

In [4]:
combine_features('data/test_Set/true/121147').shape

(8, 11, 11)

In [5]:
true_samples = np.array([combine_features(sample) for sample in glob.glob('data/test_Set/true/*')])
true_samples.shape

(207, 8, 11, 11)

In [6]:
false_samples = np.array([combine_features(sample) for sample in glob.glob('data/test_Set/false/*') if combine_features(sample).shape == (8, 11, 11)])
false_samples.shape

(271, 8, 11, 11)

In [7]:
true_samples.shape

(207, 8, 11, 11)

In [8]:
labels_false = np.zeros(271)
labels_true = np.ones(207)

In [9]:
train_true = true_samples

In [10]:
data = np.concatenate([true_samples, false_samples])

In [11]:
labels = np.concatenate([labels_true, labels_false])

In [12]:
data.shape

(478, 8, 11, 11)

In [13]:
labels.shape

(478,)

In [14]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.20, random_state=42)

/home/austin/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [15]:
X_train.shape

(382, 8, 11, 11)

In [16]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [17]:
model = Sequential()
model.add(Conv2D(filters=64, kernel_size=(2,2), padding='same', input_shape=(8, 11, 11)))
model.add(Activation('relu'))
model.add(Conv2D(64, (2,2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [18]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [19]:
X_dev = X_train[:10]
y_dev = y_train[:10]

In [20]:
X_dev -= X_dev.mean(axis=0) # subtract channel wise mean
X_dev /= X_dev.std(axis=0) # divide by channel wise std

In [21]:
# X_train -= X_train.mean(axis=0) # subtract channel wise mean
# X_train /= X_train.std(axis=0) # divide by channel wise std

# X_test -= X_train.mean(axis=0) # normalize by mean calculated on training set
# X_test /= X_train.std(axis=0)

In [22]:
model.fit(X_dev, y_dev, batch_size=32, epochs=40, shuffle=True)

Epoch 1/40
10/10 [==============================] - 0s - loss: 0.6877 - acc: 0.5000
Epoch 2/40
10/10 [==============================] - 0s - loss: 0.5459 - acc: 0.8000
Epoch 3/40
10/10 [==============================] - 0s - loss: 0.4724 - acc: 0.7000
Epoch 4/40
10/10 [==============================] - 0s - loss: 0.4288 - acc: 0.7000
Epoch 5/40
10/10 [==============================] - 0s - loss: 0.3965 - acc: 0.7000
Epoch 6/40
10/10 [==============================] - 0s - loss: 0.3623 - acc: 0.8000
Epoch 7/40
10/10 [==============================] - 0s - loss: 0.3260 - acc: 0.9000
Epoch 8/40
10/10 [==============================] - 0s - loss: 0.2943 - acc: 0.9000
Epoch 9/40
10/10 [==============================] - 0s - loss: 0.2695 - acc: 0.9000
Epoch 10/40
10/10 [==============================] - 0s - loss: 0.2473 - acc: 0.9000
Epoch 11/40
10/10 [==============================] - 0s - loss: 0.2230 - acc: 0.9000
Epoch 12/40
10/10 [==============================] - 0s - loss: 0.1983 - a

In [23]:
y_dev

array([ 0.,  1.,  0.,  0.,  1.,  0.,  1.,  1.,  0.,  1.])

In [24]:
model.predict_proba(X_dev)

10/10 [==============================] - 0s


array([[  1.01775368e-06],
       [  9.98670578e-01],
       [  6.35446042e-07],
       [  5.28842029e-05],
       [  9.99982476e-01],
       [  6.48473855e-03],
       [  9.93346691e-01],
       [  9.99586284e-01],
       [  3.18151753e-04],
       [  1.00000000e+00]], dtype=float32)

In [25]:
model.predict_classes(X_dev)

10/10 [==============================] - 0s


array([[0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1]], dtype=int32)